# Import packages

In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import StratifiedKFold, StratifiedShuffleSplit

# Load data

In [3]:
sub = pd.read_csv('./data/raw/submission_history.csv')

In [4]:
sub.head()

,id,pk,track_id,track_info,band_id,influencer_id,influencer_kind,influencer_feedback,decision,score
0,7312,7312,324,test tim,303,102,Label,"Bonjour, \nle track surf sur les codes ""austra...",['give feedback on your tune'],0.0
1,7313,7313,324,test tim,303,103,Radio,"Bonjour, merci pour votre envoi. Le morceau n'...",['give feedback on your tune'],0.0
2,7314,7314,324,test tim,303,104,Journalist,Le morceau est à lui tout seul une succession ...,['give feedback on your tune'],0.0
3,7315,7315,324,test tim,303,105,Channel,Très bonne pop aux airs de Tame Impala et Pond...,"['share it on social media', 'add it to a play...",1.0
4,7316,7316,324,test tim,303,106,Media,"La production est assurément excellente, mais ...",['give feedback on your tune'],0.0


# Split data

In [296]:
N_FOLDS = 5
RANDOM_SEED = 42

In [297]:
X = sub.drop(columns=['score', 'influencer_feedback', 'decision'])
y = sub.score
skf = StratifiedKFold(n_splits=N_FOLDS, shuffle=True, random_state=RANDOM_SEED)

In [298]:
skf.get_n_splits(X, y)
print(skf)

StratifiedKFold(n_splits=5, random_state=42, shuffle=True)


# Naive models

In [299]:
from sklearn.metrics import mean_squared_error

In [302]:
std_train_mean = 0.0
std_test_mean = 0.0
naive_score = 0.0
random_score =  0.0

for train_index, test_index in skf.split(X, (y * 100).astype(int)):
    
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]
    
    std_train_mean += np.std(y_train)
    std_test_mean += np.std(y_test)
    naive_score += np.sqrt(mean_squared_error([y_train.mean()] * len(y_test), y_test))
    random_score += np.sqrt(mean_squared_error(np.random.uniform(0, 1, size=len(y_test)), y_test))

std_train_mean /= N_FOLDS
std_test_mean /= N_FOLDS
naive_score /= N_FOLDS
random_score /= N_FOLDS

In [303]:
print(std_train_mean)
print(std_test_mean)
print(naive_score)
print(random_score)

0.39570093979500703
0.39570093963773195
0.3957009399899617
0.5501003287687661


# LGBM model

In [209]:
from lightgbm import LGBMRegressor

In [11]:
band = pd.read_csv('./data/raw/band_content.csv')
content = pd.read_csv('./data/raw/influencer_content.csv')

In [12]:
sub.head()

,id,pk,track_id,track_info,band_id,influencer_id,influencer_kind,influencer_feedback,decision,score
0,7312,7312,324,test tim,303,102,Label,"Bonjour, \nle track surf sur les codes ""austra...",['give feedback on your tune'],0.0
1,7313,7313,324,test tim,303,103,Radio,"Bonjour, merci pour votre envoi. Le morceau n'...",['give feedback on your tune'],0.0
2,7314,7314,324,test tim,303,104,Journalist,Le morceau est à lui tout seul une succession ...,['give feedback on your tune'],0.0
3,7315,7315,324,test tim,303,105,Channel,Très bonne pop aux airs de Tame Impala et Pond...,"['share it on social media', 'add it to a play...",1.0
4,7316,7316,324,test tim,303,106,Media,"La production est assurément excellente, mais ...",['give feedback on your tune'],0.0


In [13]:
content.head()

,id,influencer_id,description_fr,description_en,preferences_fr,preferences_en,Acid house,African music,Alternative rock,Ambient,...,Singer-songwriter,Soul,Surf rock,Synthpop,Synthwave,Techno,Traditional Music,Trap,Trip hop,Variété Française
0,96,96,"Ex-BSC NEWS, nouveau magazine culturel franc-t...","Ex-BSC NEWS, nouveau magazine culturel franc-t...",Musique Comtemporaine et Jazz,NaN,0,0,0,0,...,0,0,0,1,0,1,0,0,0,0
1,97,97,Underdog Records is a french alternative label...,Underdog Records is a french alternative label...,"Folk, soul, blues, rock&roll, indie pop","Folk, soul, blues, rock&roll, indie pop",0,0,0,0,...,1,1,0,0,0,0,0,0,0,0
2,102,102,HIGHLIFE is a music publishing company + Indep...,HIGHLIFE Recordings has a wide open philosophy...,Déjà de la maturité,Already mature and original,0,0,0,1,...,0,0,0,0,0,1,0,0,1,0
3,103,103,Nectar est une émission radio musicale et hebd...,Nectar is a weekly music radio program about f...,Folk,Folk,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,104,104,Ecrit pour Konbini et Noisey (Vice). Défricheu...,"Writes for Konbini and Noisey (Vice). Rap, Hip...",Rap,Rap,0,0,0,0,...,0,1,0,0,0,0,0,1,0,1


In [202]:
dataset = sub[['id', 'track_id', 'band_id', 'influencer_id', 'influencer_kind', 'score']].merge(
    band.drop(columns=['id', 'biography_fr', 'biography_en']),
    how='left',
    on='band_id'
).merge(
    content.drop(columns=['id', 'description_fr', 'description_en', 'preferences_fr', 'preferences_en']),
    how='left',
    on='influencer_id'
)

In [203]:
dataset.columns.tolist()

['id',
 'track_id',
 'band_id',
 'influencer_id',
 'influencer_kind',
 'score',
 'Acid house_x',
 'African music_x',
 'Alternative rock_x',
 'Ambient_x',
 'Blues_x',
 'Bossa Nova_x',
 'Chill-out_x',
 'Classical Music_x',
 'Coldwave_x',
 'Country_x',
 'Dance music_x',
 'Dance-pop_x',
 'Deep house_x',
 'Disco_x',
 'Dream Pop_x',
 'Dub_x',
 'Electro swing_x',
 'Electronic rock_x',
 'Electronica_x',
 'Electropop_x',
 'Experimental_x',
 'Experimental Jazz_x',
 'Experimental rock_x',
 'Film Music_x',
 'French house_x',
 'Funk_x',
 'Future house_x',
 'Garage rock_x',
 'Grime_x',
 'Hard rock_x',
 'Hip hop_x',
 'House music_x',
 'Indie folk_x',
 'Indie pop_x',
 'Indie rock_x',
 'Instrumental_x',
 'International Pop_x',
 'Latin music_x',
 'Lo-Fi_x',
 'Metal_x',
 'Minimal_x',
 'Modern Jazz_x',
 'New wave_x',
 'Noise rock_x',
 'Nouvelle Scène_x',
 'Nu-disco_x',
 'Pop rock_x',
 'Pop soul_x',
 'Post-punk_x',
 'Post-rock_x',
 'Progressive pop_x',
 'Progressive rock_x',
 'Psychedelic pop_x',
 'Psyched

In [204]:
dataset.influencer_kind.value_counts()

Media          35288
Radio           9872
Label           9155
Playlist        8501
Journalist      4268
Channel         4150
Booker          3405
Mentor          2288
Manager         2094
Springboard     2087
Publisher       1698
Supervisor       572
Event            328
Name: influencer_kind, dtype: int64

In [205]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
dataset['influencer_kind'] = le.fit_transform(dataset['influencer_kind'])

In [206]:
dataset.head()

,id,track_id,band_id,influencer_id,influencer_kind,score,Acid house_x,African music_x,Alternative rock_x,Ambient_x,...,Singer-songwriter_y,Soul_y,Surf rock_y,Synthpop_y,Synthwave_y,Techno_y,Traditional Music_y,Trap_y,Trip hop_y,Variété Française_y
0,7312,324,303,102,4,0.0,0,0,0,0,...,0,0,0,0,0,1,0,0,1,0
1,7313,324,303,103,10,0.0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,7314,324,303,104,3,0.0,0,0,0,0,...,0,1,0,0,0,0,0,1,0,1
3,7315,324,303,105,1,1.0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
4,7316,324,303,106,6,0.0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0


In [304]:
X, y = (
    dataset.drop(columns='score').rename(
        {'Variété Française_x': 'Variete Francaise_x', 'Variété Française_y': 'Variete Francaise_y'},
        axis=1
    ).values,
    dataset.score.values
)

In [312]:
lgbm_score =  0.0


for train_index, test_index in skf.split(X, (y * 100).astype(int)):
    
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    
    sss = StratifiedShuffleSplit(n_splits=1, test_size=0.1, random_state=RANDOM_SEED)
    tridx, vidx = next(sss.split(X_train, (y_train * 100).astype(int)))
    X_train, X_valid = X_train[tridx], X_train[vidx]
    y_train, y_valid = y_train[tridx], y_train[vidx]
    
    lgbm = LGBMRegressor(
        num_leaves=2**10,
        learning_rate=0.01,
        subsample=0.5,
        reg_alpha=10.0,
        reg_lambda=10.0,
        n_estimators=10000,
        silent=False
    )
    lgbm.fit(X_train, y_train, eval_set=(X_valid, y_valid), 
             eval_metric='mse', early_stopping_rounds=100,
             verbose=100)
    lgbm_score += np.sqrt(mean_squared_error(lgbm.predict(X_test), y_test))

lgbm_score /= N_FOLDS

print(lgbm_score)

Training until validation scores don't improve for 100 rounds
[100]	valid_0's l2: 0.102588
[200]	valid_0's l2: 0.0920579
[300]	valid_0's l2: 0.0889158
[400]	valid_0's l2: 0.0874495
[500]	valid_0's l2: 0.086519
[600]	valid_0's l2: 0.0858807
[700]	valid_0's l2: 0.0854668
[800]	valid_0's l2: 0.0851732
[900]	valid_0's l2: 0.0849352
[1000]	valid_0's l2: 0.0847728
[1100]	valid_0's l2: 0.0846168
[1200]	valid_0's l2: 0.0845244
[1300]	valid_0's l2: 0.0844236
[1400]	valid_0's l2: 0.0843611
[1500]	valid_0's l2: 0.0843134
[1600]	valid_0's l2: 0.0842582
[1700]	valid_0's l2: 0.0842047
[1800]	valid_0's l2: 0.0841595
[1900]	valid_0's l2: 0.0841402
Early stopping, best iteration is:
[1860]	valid_0's l2: 0.0841402
Training until validation scores don't improve for 100 rounds
[100]	valid_0's l2: 0.10309
[200]	valid_0's l2: 0.0930311
[300]	valid_0's l2: 0.0901051
[400]	valid_0's l2: 0.0888514
[500]	valid_0's l2: 0.0880205
[600]	valid_0's l2: 0.0874837
[700]	valid_0's l2: 0.0870172
[800]	valid_0's l2: 0.08

# First Keras models

In [150]:
from keras.layers import Input, Embedding, Flatten, Dot, Dense, Concatenate, Dropout
from keras.models import Model

In [ ]:
book_input = Input(shape=[1], name="Book-Input")
book_embedding = Embedding(n_books+1, 5, name="Book-Embedding")(book_input)
book_vec = Flatten(name="Flatten-Books")(book_embedding)user_input = Input(shape=[1], name="User-Input")
user_embedding = Embedding(n_users+1, 5, name="User-Embedding")(user_input)
user_vec = Flatten(name="Flatten-Users")(user_embedding)prod = Dot(name="Dot-Product", axes=1)([book_vec, user_vec])
model = Model([user_input, book_input], prod)
model.compile('adam', 'mean_squared_error')

In [22]:
dataset.head()

,id,track_id,band_id,influencer_id,influencer_kind,score,Acid house_x,African music_x,Alternative rock_x,Ambient_x,...,Singer-songwriter_y,Soul_y,Surf rock_y,Synthpop_y,Synthwave_y,Techno_y,Traditional Music_y,Trap_y,Trip hop_y,Variété Française_y
0,7312,324,303,102,4,0.0,0,0,0,0,...,0,0,0,0,0,1,0,0,1,0
1,7313,324,303,103,10,0.0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,7314,324,303,104,3,0.0,0,0,0,0,...,0,1,0,0,0,0,0,1,0,1
3,7315,324,303,105,1,1.0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
4,7316,324,303,106,6,0.0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0


In [314]:
i_data = dataset.filter(regex='_y')
b_data = dataset.filter(regex='_x')

In [315]:
def build_model(i_emb_dim=50, b_emb_dim=50):
    """
    """
    influencer_input = Input(shape=[i_data.shape[1]], name="Influencer-Input")
    influencer_embedding = Dense(i_emb_dim, activation='tanh', name="Influencer-Embedding")(influencer_input)
    
    band_input = Input(shape=[b_data.shape[1]], name="Band-Input")
    band_embedding = Dense(b_emb_dim, activation='tanh', name="Band-Embedding")(band_input)
    
    prod = Dot(name="Dot-Product", axes=1)([influencer_embedding, band_embedding])
    model = Model([influencer_input, band_input], prod)
    model.compile('adam', 'mean_squared_error')
    
    return model

In [37]:
model = build_model()

In [38]:
history = model.fit([i_data, b_data], dataset.score, epochs=10, verbose=1)



Epoch 1/10
83706/83706 [==============================] - 6s 73us/step - loss: 0.1854
Epoch 2/10
83706/83706 [==============================] - 4s 51us/step - loss: 0.1366
Epoch 3/10
83706/83706 [==============================] - 4s 52us/step - loss: 0.1264
Epoch 4/10
83706/83706 [==============================] - 4s 51us/step - loss: 0.1195
Epoch 5/10
83706/83706 [==============================] - 4s 52us/step - loss: 0.1158
Epoch 6/10
83706/83706 [==============================] - 4s 52us/step - loss: 0.1131
Epoch 7/10
83706/83706 [==============================] - 4s 52us/step - loss: 0.1099
Epoch 8/10
83706/83706 [==============================] - 4s 52us/step - loss: 0.1086
Epoch 9/10
83706/83706 [==============================] - 4s 53us/step - loss: 0.1067
Epoch 10/10
83706/83706 [==============================] - 5s 54us/step - loss: 0.1059


In [39]:
# model.save('regression_model.h5')

In [316]:
X, y = (
    dataset.drop(columns='score').rename(
        {'Variété Française_x': 'Variete Francaise_x', 'Variété Française_y': 'Variete Francaise_y'},
        axis=1
    ).values,
    dataset.score.values
)

In [317]:
i_data = dataset.filter(regex='_y')
b_data = dataset.filter(regex='_x')

In [318]:
i_data_idx = [dataset.drop(columns='score').columns.get_loc(c) for c in dataset.filter(regex='_y')]
b_data_idx = [dataset.drop(columns='score').columns.get_loc(c) for c in dataset.filter(regex='_x')]

In [319]:
dataset.columns.get_loc('Variété Française_y')

147

In [320]:
dataset.shape

(83706, 148)

In [321]:
nn_score = 0.0

for train_index, test_index in skf.split(X, (y * 100).astype(int)):
    
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    
    sss = StratifiedShuffleSplit(n_splits=1, test_size=0.3, random_state=RANDOM_SEED)
    tridx, vidx = next(sss.split(X_train, (y_train * 100).astype(int)))
    X_train, X_valid = X_train[tridx], X_train[vidx]
    y_train, y_valid = y_train[tridx], y_train[vidx]
    
    model = build_model()
    model.fit([X_train[:, i_data_idx], X_train[:, b_data_idx]], y_train,
              validation_data=([X_valid[:, i_data_idx], X_valid[:, b_data_idx]], y_valid), 
              batch_size=64,
              epochs=10,
              verbose=1)
    
    nn_score += np.sqrt(mean_squared_error(model.predict([X_test[:, i_data_idx], X_test[:, b_data_idx]]), y_test))

nn_score /= N_FOLDS

print(nn_score)

Train on 46874 samples, validate on 20090 samples
Epoch 1/10
46874/46874 [==============================] - 7s 150us/step - loss: 0.2231 - val_loss: 0.1751
Epoch 2/10
46874/46874 [==============================] - 4s 75us/step - loss: 0.1512 - val_loss: 0.1471
Epoch 3/10
46874/46874 [==============================] - 4s 76us/step - loss: 0.1372 - val_loss: 0.1382
Epoch 4/10
46874/46874 [==============================] - 4s 75us/step - loss: 0.1299 - val_loss: 0.1347
Epoch 5/10
46874/46874 [==============================] - 4s 76us/step - loss: 0.1240 - val_loss: 0.1358
Epoch 6/10
46874/46874 [==============================] - 4s 75us/step - loss: 0.1208 - val_loss: 0.1293
Epoch 7/10
46874/46874 [==============================] - 4s 75us/step - loss: 0.1175 - val_loss: 0.1253
Epoch 8/10
46874/46874 [==============================] - 4s 76us/step - loss: 0.1153 - val_loss: 0.1221
Epoch 9/10
46874/46874 [==============================] - 4s 77us/step - loss: 0.1129 - val_loss: 0.1193
Epoc

In [73]:
dataset.influencer_kind.nunique()

13

In [322]:
def build_model_2(i_emb_dim=50, b_emb_dim=50, kind_emb_dim=5, last_dense=20, dropout=0.2):
    """
    """
    # Influencer embedding
    influencer_input = Input(shape=[i_data.shape[1]], name="Influencer-Input")
    influencer_embedding = Dense(i_emb_dim, activation='tanh', name="Influencer-Embedding")(influencer_input)
    
    # Influencer kind categorical embedding
    influencer_kind_input = Input(shape=[1], name="Influencer-Kind-Input")
    influencer_kind_emb = Embedding(14, kind_emb_dim, name="Influencer-Kind-Embedding")(influencer_kind_input)
    
    # Concatenate influencer emb with influencer kind emb to get full influencer emb
    influencer_full_emb = Concatenate(axis=-1)([influencer_embedding, Flatten(name='Flatten')(influencer_kind_emb)])
    
    # Band embedding
    band_input = Input(shape=[b_data.shape[1]], name="Band-Input")
    band_embedding = Dense(b_emb_dim, activation='tanh', name="Band-Embedding")(band_input)
    
    # Concatenate and create product
    prod = Concatenate(name="Concat", axis=-1)([influencer_full_emb, band_embedding])
    prod2 = Dense(last_dense, activation='tanh', name="Dense1")(prod)
    dropout = Dropout(rate=dropout)(prod2)
    
    # Dropout
    prod3 = Dense(1, activation='tanh', name="Dense2")(dropout)
    model = Model([influencer_input, band_input, influencer_kind_input], prod3)
    model.compile('adam', 'mean_squared_error')
    
    return model

In [323]:
def build_model_3(i_emb_dim=60, b_emb_dim=60, kind_emb_dim=10, last_dense=40, dropout=0.3):
    """
    """
    # Influencer embedding
    influencer_input = Input(shape=[i_data.shape[1]], name="Influencer-Input")
    influencer_embedding = Dense(i_emb_dim, activation='relu', name="Influencer-Embedding1")(influencer_input)
    influencer_embedding = Dense(last_dense, activation='relu', name="Dense1")(influencer_embedding)
    influencer_embedding = Dense(i_emb_dim-10, activation='relu', name="Influencer-Embedding")(influencer_embedding)
    
    # Influencer kind categorical embedding
    influencer_kind_input = Input(shape=[1], name="Influencer-Kind-Input")
    influencer_kind_emb = Embedding(14, kind_emb_dim, name="Influencer-Kind-Embedding")(influencer_kind_input)
    
    # Concatenate influencer emb with influencer kind emb to get full influencer emb
    influencer_full_emb = Concatenate(axis=-1)([influencer_embedding, Flatten(name='Flatten')(influencer_kind_emb)])
    
    # Band embedding
    band_input = Input(shape=[b_data.shape[1]], name="Band-Input")
    band_embedding = Dense(b_emb_dim, activation='relu', name="Band-Embedding1")(band_input)
    band_embedding = Dense(last_dense, activation='relu', name="Dense2")(band_embedding)
    band_embedding = Dense(b_emb_dim-10, activation='relu', name="Band-Embedding")(band_embedding)
    
    # Concatenate and create product
    prod = Concatenate(name="Concat", axis=-1)([influencer_full_emb, band_embedding])
    prod2 = Dense(last_dense, activation='relu', name="Dense0")(prod)
    dropout = Dropout(rate=dropout)(prod2)
    
    # Dropout
    prod3 = Dense(1, activation='relu', name="Dense3")(dropout)
    model = Model([influencer_input, band_input, influencer_kind_input], prod3)
    model.compile('adam', 'mean_squared_error')
    
    return model

In [324]:
dataset.head()

,id,track_id,band_id,influencer_id,influencer_kind,score,Acid house_x,African music_x,Alternative rock_x,Ambient_x,...,Singer-songwriter_y,Soul_y,Surf rock_y,Synthpop_y,Synthwave_y,Techno_y,Traditional Music_y,Trap_y,Trip hop_y,Variété Française_y
0,7312,324,303,102,4,0.0,0,0,0,0,...,0,0,0,0,0,1,0,0,1,0
1,7313,324,303,103,10,0.0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,7314,324,303,104,3,0.0,0,0,0,0,...,0,1,0,0,0,0,0,1,0,1
3,7315,324,303,105,1,1.0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
4,7316,324,303,106,6,0.0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0


In [333]:
EPOCHS = 10000
PATIENCE = 50
BATCH_SIZE = 32

In [334]:
nn_score = 0.0

for train_index, test_index in skf.split(X, (y * 100).astype(int)):
    
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    
    sss = StratifiedShuffleSplit(n_splits=1, test_size=0.3, random_state=RANDOM_SEED)
    tridx, vidx = next(sss.split(X_train, (y_train * 100).astype(int)))
    X_train, X_valid = X_train[tridx], X_train[vidx]
    y_train, y_valid = y_train[tridx], y_train[vidx]
    
    # Build model
    model = build_model_3()
    
    # Early stoppnig callback
    es = keras.callbacks.EarlyStopping(
        monitor='val_loss', 
        mode='min', 
        patience=PATIENCE,
        restore_best_weights=True,
        verbose=1
    )
    
    # Fit
    model.fit([X_train[:, i_data_idx], X_train[:, b_data_idx], X_train[:, 5]], y_train,
              validation_data=([X_valid[:, i_data_idx], X_valid[:, b_data_idx], X_valid[:, 5]], y_valid), 
              batch_size=BATCH_SIZE,
              epochs=EPOCHS,
              callbacks=[es],
              verbose=1)
    
    nn_score += np.sqrt(mean_squared_error(
        model.predict([X_test[:, i_data_idx], X_test[:, b_data_idx], X_test[:, 5]]), y_test
    ))

nn_score /= N_FOLDS

print(nn_score)

Train on 46874 samples, validate on 20090 samples
Epoch 1/10000
46874/46874 [==============================] - 16s 334us/step - loss: 0.1167 - val_loss: 0.1042
Epoch 2/10000
46874/46874 [==============================] - 11s 232us/step - loss: 0.1032 - val_loss: 0.0989
Epoch 3/10000
46874/46874 [==============================] - 11s 229us/step - loss: 0.1003 - val_loss: 0.0977
Epoch 4/10000
46874/46874 [==============================] - 11s 238us/step - loss: 0.0993 - val_loss: 0.0954
Epoch 5/10000
46874/46874 [==============================] - 11s 237us/step - loss: 0.0974 - val_loss: 0.0975
Epoch 6/10000
46874/46874 [==============================] - 11s 231us/step - loss: 0.0965 - val_loss: 0.0953
Epoch 7/10000
46874/46874 [==============================] - 11s 230us/step - loss: 0.0959 - val_loss: 0.0949
Epoch 8/10000
46874/46874 [==============================] - 11s 232us/step - loss: 0.0951 - val_loss: 0.0945
Epoch 9/10000
46874/46874 [==============================] - 10s 223us

46875/46875 [==============================] - 11s 227us/step - loss: 0.0934 - val_loss: 0.0967
Epoch 12/10000
46875/46875 [==============================] - 11s 227us/step - loss: 0.0935 - val_loss: 0.0965
Epoch 13/10000
46875/46875 [==============================] - 11s 227us/step - loss: 0.0928 - val_loss: 0.0969
Epoch 14/10000
46875/46875 [==============================] - 11s 227us/step - loss: 0.0923 - val_loss: 0.0957
Epoch 15/10000
46875/46875 [==============================] - 12s 245us/step - loss: 0.0917 - val_loss: 0.0959
Epoch 16/10000
46875/46875 [==============================] - 12s 248us/step - loss: 0.0916 - val_loss: 0.0965
Epoch 17/10000
46875/46875 [==============================] - 12s 256us/step - loss: 0.0913 - val_loss: 0.0968
Epoch 18/10000
46875/46875 [==============================] - 11s 241us/step - loss: 0.0908 - val_loss: 0.0968
Epoch 19/10000
46875/46875 [==============================] - 11s 242us/step - loss: 0.0907 - val_loss: 0.0971
Epoch 20/10000
4

Epoch 20/10000
46875/46875 [==============================] - 11s 233us/step - loss: 0.0903 - val_loss: 0.0973
Epoch 21/10000
46875/46875 [==============================] - 11s 232us/step - loss: 0.0898 - val_loss: 0.0979
Epoch 22/10000
46875/46875 [==============================] - 11s 233us/step - loss: 0.0898 - val_loss: 0.0968
Epoch 23/10000
46875/46875 [==============================] - 11s 232us/step - loss: 0.0894 - val_loss: 0.0980
Epoch 24/10000
46875/46875 [==============================] - 11s 232us/step - loss: 0.0892 - val_loss: 0.0977
Epoch 25/10000
46875/46875 [==============================] - 11s 233us/step - loss: 0.0888 - val_loss: 0.0970
Epoch 26/10000
46875/46875 [==============================] - 11s 232us/step - loss: 0.0887 - val_loss: 0.0970
Epoch 27/10000
46875/46875 [==============================] - 11s 231us/step - loss: 0.0882 - val_loss: 0.0968
Epoch 28/10000
46875/46875 [==============================] - 11s 234us/step - loss: 0.0884 - val_loss: 0.0969
E

46875/46875 [==============================] - 11s 238us/step - loss: 0.0859 - val_loss: 0.0980
Epoch 34/10000
46875/46875 [==============================] - 11s 237us/step - loss: 0.0857 - val_loss: 0.0987
Epoch 35/10000
46875/46875 [==============================] - 11s 236us/step - loss: 0.0853 - val_loss: 0.1000
Epoch 36/10000
46875/46875 [==============================] - 11s 236us/step - loss: 0.0855 - val_loss: 0.0988
Epoch 37/10000
46875/46875 [==============================] - 11s 236us/step - loss: 0.0851 - val_loss: 0.0988
Epoch 38/10000
46875/46875 [==============================] - 11s 236us/step - loss: 0.0852 - val_loss: 0.0994
Epoch 39/10000
46875/46875 [==============================] - 11s 237us/step - loss: 0.0850 - val_loss: 0.0996
Epoch 40/10000
46875/46875 [==============================] - 11s 237us/step - loss: 0.0849 - val_loss: 0.1000
Epoch 41/10000
46875/46875 [==============================] - 11s 236us/step - loss: 0.0845 - val_loss: 0.1003
Epoch 42/10000
4

Epoch 40/10000
46875/46875 [==============================] - 11s 238us/step - loss: 0.0845 - val_loss: 0.0991
Epoch 41/10000
46875/46875 [==============================] - 11s 239us/step - loss: 0.0845 - val_loss: 0.0982
Epoch 42/10000
46875/46875 [==============================] - 11s 239us/step - loss: 0.0846 - val_loss: 0.0998
Epoch 43/10000
46875/46875 [==============================] - 11s 237us/step - loss: 0.0846 - val_loss: 0.0979
Epoch 44/10000
46875/46875 [==============================] - 11s 239us/step - loss: 0.0837 - val_loss: 0.0980
Epoch 45/10000
46875/46875 [==============================] - 11s 238us/step - loss: 0.0838 - val_loss: 0.1009
Epoch 46/10000
46875/46875 [==============================] - 11s 239us/step - loss: 0.0839 - val_loss: 0.0994
Epoch 47/10000
46875/46875 [==============================] - 11s 239us/step - loss: 0.0831 - val_loss: 0.0996
Epoch 48/10000
46875/46875 [==============================] - 11s 239us/step - loss: 0.0832 - val_loss: 0.0980
E

In [335]:
nn_score = 0.0

for train_index, test_index in skf.split(X, (y * 100).astype(int)):
    
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    
    sss = StratifiedShuffleSplit(n_splits=1, test_size=0.3, random_state=RANDOM_SEED)
    tridx, vidx = next(sss.split(X_train, (y_train * 100).astype(int)))
    X_train, X_valid = X_train[tridx], X_train[vidx]
    y_train, y_valid = y_train[tridx], y_train[vidx]
    
    # Build model
    model = build_model_3(80, 80, 5, 50, 0.3)
    
    # Early stoppnig callback
    es = keras.callbacks.EarlyStopping(
        monitor='val_loss', 
        mode='min', 
        patience=PATIENCE,
        restore_best_weights=True,
        verbose=1
    )
    
    # Fit
    model.fit([X_train[:, i_data_idx], X_train[:, b_data_idx], X_train[:, 5]], y_train,
              validation_data=([X_valid[:, i_data_idx], X_valid[:, b_data_idx], X_valid[:, 5]], y_valid), 
              batch_size=BATCH_SIZE,
              epochs=EPOCHS,
              callbacks=[es],
              verbose=1)
    
    nn_score += np.sqrt(mean_squared_error(
        model.predict([X_test[:, i_data_idx], X_test[:, b_data_idx], X_test[:, 5]]), y_test
    ))

nn_score /= N_FOLDS

print(nn_score)

Train on 46874 samples, validate on 20090 samples
Epoch 1/10000
46874/46874 [==============================] - 16s 346us/step - loss: 0.1133 - val_loss: 0.0995
Epoch 2/10000
46874/46874 [==============================] - 11s 242us/step - loss: 0.1032 - val_loss: 0.0976
Epoch 3/10000
46874/46874 [==============================] - 11s 242us/step - loss: 0.0996 - val_loss: 0.0965
Epoch 4/10000
46874/46874 [==============================] - 11s 242us/step - loss: 0.0985 - val_loss: 0.0965
Epoch 5/10000
46874/46874 [==============================] - 11s 242us/step - loss: 0.0970 - val_loss: 0.0953
Epoch 6/10000
46874/46874 [==============================] - 11s 243us/step - loss: 0.0962 - val_loss: 0.0978
Epoch 7/10000
46874/46874 [==============================] - 11s 243us/step - loss: 0.0955 - val_loss: 0.0945
Epoch 8/10000
46874/46874 [==============================] - 11s 244us/step - loss: 0.0947 - val_loss: 0.0953
Epoch 9/10000
46874/46874 [==============================] - 11s 243us

46875/46875 [==============================] - 11s 243us/step - loss: 0.0914 - val_loss: 0.0970
Epoch 15/10000
46875/46875 [==============================] - 11s 244us/step - loss: 0.0910 - val_loss: 0.0972
Epoch 16/10000
46875/46875 [==============================] - 11s 245us/step - loss: 0.0908 - val_loss: 0.0960
Epoch 17/10000
46875/46875 [==============================] - 11s 243us/step - loss: 0.0903 - val_loss: 0.0956
Epoch 18/10000
46875/46875 [==============================] - 11s 244us/step - loss: 0.0899 - val_loss: 0.0968
Epoch 19/10000
46875/46875 [==============================] - 11s 244us/step - loss: 0.0900 - val_loss: 0.0978
Epoch 20/10000
46875/46875 [==============================] - 11s 243us/step - loss: 0.0896 - val_loss: 0.0973
Epoch 21/10000
46875/46875 [==============================] - 11s 245us/step - loss: 0.0890 - val_loss: 0.0980
Epoch 22/10000
46875/46875 [==============================] - 11s 244us/step - loss: 0.0885 - val_loss: 0.0982
Epoch 23/10000
4

Epoch 20/10000
46875/46875 [==============================] - 15s 309us/step - loss: 0.0899 - val_loss: 0.0978
Epoch 21/10000
46875/46875 [==============================] - 15s 312us/step - loss: 0.0892 - val_loss: 0.0970
Epoch 22/10000
46875/46875 [==============================] - 15s 310us/step - loss: 0.0894 - val_loss: 0.0977
Epoch 23/10000
46875/46875 [==============================] - 15s 311us/step - loss: 0.0888 - val_loss: 0.0977
Epoch 24/10000
46875/46875 [==============================] - 15s 311us/step - loss: 0.0885 - val_loss: 0.0981
Epoch 25/10000
46875/46875 [==============================] - 15s 310us/step - loss: 0.0885 - val_loss: 0.0971
Epoch 26/10000
46875/46875 [==============================] - 15s 311us/step - loss: 0.0877 - val_loss: 0.0982
Epoch 27/10000
46875/46875 [==============================] - 15s 311us/step - loss: 0.0880 - val_loss: 0.0986
Epoch 28/10000
46875/46875 [==============================] - 15s 311us/step - loss: 0.0876 - val_loss: 0.0985
E

46875/46875 [==============================] - 12s 249us/step - loss: 0.0853 - val_loss: 0.0981
Epoch 31/10000
46875/46875 [==============================] - 12s 249us/step - loss: 0.0857 - val_loss: 0.0987
Epoch 32/10000
46875/46875 [==============================] - 12s 250us/step - loss: 0.0851 - val_loss: 0.0991
Epoch 33/10000
46875/46875 [==============================] - 12s 249us/step - loss: 0.0848 - val_loss: 0.0992
Epoch 34/10000
46875/46875 [==============================] - 12s 249us/step - loss: 0.0847 - val_loss: 0.0985
Epoch 35/10000
46875/46875 [==============================] - 12s 249us/step - loss: 0.0845 - val_loss: 0.0999
Epoch 36/10000
46875/46875 [==============================] - 12s 249us/step - loss: 0.0841 - val_loss: 0.1005
Epoch 37/10000
46875/46875 [==============================] - 12s 249us/step - loss: 0.0840 - val_loss: 0.1011
Epoch 38/10000
46875/46875 [==============================] - 12s 249us/step - loss: 0.0841 - val_loss: 0.1006
Epoch 39/10000
4

Epoch 41/10000
46875/46875 [==============================] - 12s 254us/step - loss: 0.0833 - val_loss: 0.0976
Epoch 42/10000
46875/46875 [==============================] - 12s 254us/step - loss: 0.0835 - val_loss: 0.0977
Epoch 43/10000
46875/46875 [==============================] - 12s 254us/step - loss: 0.0829 - val_loss: 0.0997
Epoch 44/10000
46875/46875 [==============================] - 12s 254us/step - loss: 0.0829 - val_loss: 0.0987
Epoch 45/10000
46875/46875 [==============================] - 12s 254us/step - loss: 0.0826 - val_loss: 0.0987
Epoch 46/10000
46875/46875 [==============================] - 12s 254us/step - loss: 0.0823 - val_loss: 0.0987
Epoch 47/10000
46875/46875 [==============================] - 12s 254us/step - loss: 0.0822 - val_loss: 0.0990
Epoch 48/10000
46875/46875 [==============================] - 12s 254us/step - loss: 0.0823 - val_loss: 0.0993
Epoch 49/10000
46875/46875 [==============================] - 12s 253us/step - loss: 0.0818 - val_loss: 0.0986
E

In [336]:
nn_score = 0.0

for train_index, test_index in skf.split(X, (y * 100).astype(int)):
    
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    
    sss = StratifiedShuffleSplit(n_splits=1, test_size=0.3, random_state=RANDOM_SEED)
    tridx, vidx = next(sss.split(X_train, (y_train * 100).astype(int)))
    X_train, X_valid = X_train[tridx], X_train[vidx]
    y_train, y_valid = y_train[tridx], y_train[vidx]
    
    # Build model
    model = build_model_3(80, 80, 5, 50, 0.5)
    
    # Early stoppnig callback
    es = keras.callbacks.EarlyStopping(
        monitor='val_loss', 
        mode='min', 
        patience=PATIENCE,
        restore_best_weights=True,
        verbose=1
    )
    
    # Fit
    model.fit([X_train[:, i_data_idx], X_train[:, b_data_idx], X_train[:, 5]], y_train,
              validation_data=([X_valid[:, i_data_idx], X_valid[:, b_data_idx], X_valid[:, 5]], y_valid), 
              batch_size=BATCH_SIZE,
              epochs=EPOCHS,
              callbacks=[es],
              verbose=1)
    
    nn_score += np.sqrt(mean_squared_error(
        model.predict([X_test[:, i_data_idx], X_test[:, b_data_idx], X_test[:, 5]]), y_test
    ))

nn_score /= N_FOLDS

print(nn_score)

Train on 46874 samples, validate on 20090 samples
Epoch 1/10000
46874/46874 [==============================] - 17s 367us/step - loss: 0.1194 - val_loss: 0.1088
Epoch 2/10000
46874/46874 [==============================] - 12s 255us/step - loss: 0.1048 - val_loss: 0.0991
Epoch 3/10000
46874/46874 [==============================] - 12s 256us/step - loss: 0.1018 - val_loss: 0.0973
Epoch 4/10000
46874/46874 [==============================] - 12s 255us/step - loss: 0.1000 - val_loss: 0.1009
Epoch 5/10000
46874/46874 [==============================] - 12s 256us/step - loss: 0.0990 - val_loss: 0.0964
Epoch 6/10000
46874/46874 [==============================] - 12s 255us/step - loss: 0.0984 - val_loss: 0.0956
Epoch 7/10000
46874/46874 [==============================] - 12s 256us/step - loss: 0.0971 - val_loss: 0.0955
Epoch 8/10000
46874/46874 [==============================] - 12s 256us/step - loss: 0.0963 - val_loss: 0.0952
Epoch 9/10000
46874/46874 [==============================] - 12s 257us

46875/46875 [==============================] - 17s 368us/step - loss: 0.1202 - val_loss: 0.1005
Epoch 2/10000
46875/46875 [==============================] - 12s 256us/step - loss: 0.1033 - val_loss: 0.0989
Epoch 3/10000
46875/46875 [==============================] - 12s 255us/step - loss: 0.1009 - val_loss: 0.1132
Epoch 4/10000
46875/46875 [==============================] - 12s 254us/step - loss: 0.0996 - val_loss: 0.0977
Epoch 5/10000
46875/46875 [==============================] - 12s 256us/step - loss: 0.0981 - val_loss: 0.0975
Epoch 6/10000
46875/46875 [==============================] - 12s 256us/step - loss: 0.0970 - val_loss: 0.0977
Epoch 7/10000
46875/46875 [==============================] - 12s 257us/step - loss: 0.0966 - val_loss: 0.0970
Epoch 8/10000
46875/46875 [==============================] - 12s 255us/step - loss: 0.0955 - val_loss: 0.0965
Epoch 9/10000
46875/46875 [==============================] - 12s 255us/step - loss: 0.0949 - val_loss: 0.0963
Epoch 10/10000
46875/468

Epoch 15/10000
46875/46875 [==============================] - 12s 259us/step - loss: 0.0933 - val_loss: 0.0958
Epoch 16/10000
46875/46875 [==============================] - 12s 259us/step - loss: 0.0925 - val_loss: 0.0968
Epoch 17/10000
46875/46875 [==============================] - 12s 261us/step - loss: 0.0925 - val_loss: 0.0966
Epoch 18/10000
46875/46875 [==============================] - 12s 259us/step - loss: 0.0921 - val_loss: 0.0974
Epoch 19/10000
46875/46875 [==============================] - 12s 260us/step - loss: 0.0916 - val_loss: 0.0964
Epoch 20/10000
46875/46875 [==============================] - 12s 260us/step - loss: 0.0914 - val_loss: 0.0959
Epoch 21/10000
46875/46875 [==============================] - 12s 261us/step - loss: 0.0914 - val_loss: 0.0969
Epoch 22/10000
46875/46875 [==============================] - 12s 260us/step - loss: 0.0909 - val_loss: 0.0972
Epoch 23/10000
46875/46875 [==============================] - 12s 259us/step - loss: 0.0908 - val_loss: 0.0971
E

46875/46875 [==============================] - 12s 261us/step - loss: 0.0898 - val_loss: 0.0984
Epoch 26/10000
46875/46875 [==============================] - 12s 261us/step - loss: 0.0901 - val_loss: 0.0987
Epoch 27/10000
46875/46875 [==============================] - 12s 260us/step - loss: 0.0896 - val_loss: 0.0987
Epoch 28/10000
46875/46875 [==============================] - 12s 261us/step - loss: 0.0896 - val_loss: 0.0980
Epoch 29/10000
46875/46875 [==============================] - 12s 261us/step - loss: 0.0898 - val_loss: 0.0981
Epoch 30/10000
46875/46875 [==============================] - 12s 260us/step - loss: 0.0895 - val_loss: 0.0988
Epoch 31/10000
46875/46875 [==============================] - 12s 260us/step - loss: 0.0890 - val_loss: 0.0988
Epoch 32/10000
46875/46875 [==============================] - 12s 260us/step - loss: 0.0888 - val_loss: 0.0980
Epoch 33/10000
46875/46875 [==============================] - 12s 261us/step - loss: 0.0886 - val_loss: 0.0985
Epoch 34/10000
4

Epoch 38/10000
46875/46875 [==============================] - 12s 263us/step - loss: 0.0880 - val_loss: 0.0980
Epoch 39/10000
46875/46875 [==============================] - 12s 263us/step - loss: 0.0881 - val_loss: 0.0974
Epoch 40/10000
46875/46875 [==============================] - 12s 264us/step - loss: 0.0875 - val_loss: 0.0997
Epoch 41/10000
46875/46875 [==============================] - 12s 264us/step - loss: 0.0875 - val_loss: 0.0981
Epoch 42/10000
46875/46875 [==============================] - 12s 263us/step - loss: 0.0875 - val_loss: 0.0984
Epoch 43/10000
46875/46875 [==============================] - 12s 264us/step - loss: 0.0872 - val_loss: 0.0989
Epoch 44/10000
46875/46875 [==============================] - 12s 263us/step - loss: 0.0875 - val_loss: 0.0989
Epoch 45/10000
46875/46875 [==============================] - 12s 264us/step - loss: 0.0869 - val_loss: 0.0983
Epoch 46/10000
46875/46875 [==============================] - 12s 266us/step - loss: 0.0867 - val_loss: 0.0979
E

* Check pipe
* Add L2 reg
* look for text embeddings
* preprocess text
* build archi
* print archi
* pretrained emb
* tqdm notebook

In [359]:
def build_model_3(i_emb_dim=60, b_emb_dim=60, kind_emb_dim=10, last_dense=40, dropout=0.3):
    """
    """
    # Influencer embedding
    influencer_input = Input(shape=[i_data.shape[1]], name="Influencer-Input")
    influencer_embedding = Dense(i_emb_dim, activation='relu', name="Influencer-Embedding1")(influencer_input)
    influencer_embedding = Dense(last_dense, activation='relu', name="Dense1")(influencer_embedding)
    influencer_embedding = Dense(i_emb_dim-10, activation='relu', name="Influencer-Embedding")(influencer_embedding)
    
    # Influencer kind categorical embedding
    influencer_kind_input = Input(shape=[1], name="Influencer-Kind-Input")
    influencer_kind_emb = Embedding(14, kind_emb_dim, name="Influencer-Kind-Embedding")(influencer_kind_input)
    
    # Concatenate influencer emb with influencer kind emb to get full influencer emb
    influencer_full_emb = Concatenate(axis=-1)([influencer_embedding, Flatten(name='Flatten')(influencer_kind_emb)])
    
    # Band embedding
    band_input = Input(shape=[b_data.shape[1]], name="Band-Input")
    band_embedding = Dense(b_emb_dim, activation='relu', name="Band-Embedding1")(band_input)
    band_embedding = Dense(last_dense, activation='relu', name="Dense2")(band_embedding)
    band_embedding = Dense(b_emb_dim-10, activation='relu', name="Band-Embedding")(band_embedding)
    
    # Concatenate and create product
    prod = Concatenate(name="Concat", axis=-1)([influencer_full_emb, band_embedding])
    prod2 = Dense(last_dense, activation='relu', name="Dense0")(prod)
    dropout = Dropout(rate=dropout)(prod2)
    prod_ = Dense(50, activation='relu', name="Dense_")(prod2)
    drop = Dropout(rate=0.3)(prod_)
    
    # Dropout
    prod3 = Dense(1, activation='relu', name="Dense3")(drop)
    model = Model([influencer_input, band_input, influencer_kind_input], prod3)
    model.compile('adam', 'mean_squared_error')
    
    return model

In [360]:
nn_score = 0.0

for train_index, test_index in skf.split(X, (y * 100).astype(int)):
    
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    
    sss = StratifiedShuffleSplit(n_splits=1, test_size=0.3, random_state=RANDOM_SEED)
    tridx, vidx = next(sss.split(X_train, (y_train * 100).astype(int)))
    X_train, X_valid = X_train[tridx], X_train[vidx]
    y_train, y_valid = y_train[tridx], y_train[vidx]
    
    # Build model
    model = build_model_3(80, 80, 8, 100, 0.3)
    
    # Early stoppnig callback
    es = keras.callbacks.EarlyStopping(
        monitor='val_loss', 
        mode='min', 
        patience=PATIENCE,
        restore_best_weights=True,
        verbose=1
    )
    
    # Fit
    model.fit([X_train[:, i_data_idx], X_train[:, b_data_idx], X_train[:, 5]], y_train,
              validation_data=([X_valid[:, i_data_idx], X_valid[:, b_data_idx], X_valid[:, 5]], y_valid), 
              batch_size=BATCH_SIZE,
              epochs=EPOCHS,
              callbacks=[es],
              verbose=1)
    
    nn_score += np.sqrt(mean_squared_error(
        model.predict([X_test[:, i_data_idx], X_test[:, b_data_idx], X_test[:, 5]]), y_test
    ))

nn_score /= N_FOLDS

print(nn_score)

Train on 46874 samples, validate on 20090 samples
Epoch 1/10000
46874/46874 [==============================] - 21s 438us/step - loss: 0.1130 - val_loss: 0.1007
Epoch 2/10000
46874/46874 [==============================] - 14s 293us/step - loss: 0.1022 - val_loss: 0.0998
Epoch 3/10000
46874/46874 [==============================] - 14s 291us/step - loss: 0.0992 - val_loss: 0.0965
Epoch 4/10000
46874/46874 [==============================] - 14s 290us/step - loss: 0.0978 - val_loss: 0.0970
Epoch 5/10000
46874/46874 [==============================] - 14s 290us/step - loss: 0.0962 - val_loss: 0.0962
Epoch 6/10000
46874/46874 [==============================] - 14s 290us/step - loss: 0.0950 - val_loss: 0.0967
Epoch 7/10000
46874/46874 [==============================] - 14s 290us/step - loss: 0.0938 - val_loss: 0.0956
Epoch 8/10000
46874/46874 [==============================] - 14s 290us/step - loss: 0.0934 - val_loss: 0.0957
Epoch 9/10000
46874/46874 [==============================] - 14s 302us

46875/46875 [==============================] - 14s 293us/step - loss: 0.0862 - val_loss: 0.0986
Epoch 17/10000
46875/46875 [==============================] - 14s 291us/step - loss: 0.0850 - val_loss: 0.0983
Epoch 18/10000
46875/46875 [==============================] - 14s 291us/step - loss: 0.0844 - val_loss: 0.0988
Epoch 19/10000
46875/46875 [==============================] - 14s 291us/step - loss: 0.0838 - val_loss: 0.1012
Epoch 20/10000
46875/46875 [==============================] - 14s 291us/step - loss: 0.0826 - val_loss: 0.1006
Epoch 21/10000
46875/46875 [==============================] - 14s 291us/step - loss: 0.0818 - val_loss: 0.1016
Epoch 22/10000
46875/46875 [==============================] - 14s 291us/step - loss: 0.0814 - val_loss: 0.1009
Epoch 23/10000
46875/46875 [==============================] - 14s 291us/step - loss: 0.0807 - val_loss: 0.1019
Epoch 24/10000
46875/46875 [==============================] - 14s 291us/step - loss: 0.0797 - val_loss: 0.1014
Epoch 25/10000
4

Epoch 30/10000
46875/46875 [==============================] - 14s 294us/step - loss: 0.0735 - val_loss: 0.1061
Epoch 31/10000
46875/46875 [==============================] - 14s 293us/step - loss: 0.0721 - val_loss: 0.1083
Epoch 32/10000
46875/46875 [==============================] - 14s 294us/step - loss: 0.0712 - val_loss: 0.1057
Epoch 33/10000
46875/46875 [==============================] - 14s 293us/step - loss: 0.0708 - val_loss: 0.1093
Epoch 34/10000
46875/46875 [==============================] - 14s 294us/step - loss: 0.0702 - val_loss: 0.1069
Epoch 35/10000
46875/46875 [==============================] - 14s 294us/step - loss: 0.0694 - val_loss: 0.1096
Epoch 36/10000
46875/46875 [==============================] - 14s 293us/step - loss: 0.0688 - val_loss: 0.1098
Epoch 37/10000
46875/46875 [==============================] - 14s 292us/step - loss: 0.0683 - val_loss: 0.1118
Epoch 38/10000
46875/46875 [==============================] - 14s 294us/step - loss: 0.0680 - val_loss: 0.1110
E

46875/46875 [==============================] - 14s 298us/step - loss: 0.0664 - val_loss: 0.1125
Epoch 46/10000
46875/46875 [==============================] - 14s 297us/step - loss: 0.0659 - val_loss: 0.1113
Epoch 47/10000
46875/46875 [==============================] - 14s 296us/step - loss: 0.0649 - val_loss: 0.1129
Epoch 48/10000
46875/46875 [==============================] - 14s 296us/step - loss: 0.0649 - val_loss: 0.1122
Epoch 49/10000
46875/46875 [==============================] - 14s 297us/step - loss: 0.0643 - val_loss: 0.1161
Epoch 50/10000
46875/46875 [==============================] - 14s 298us/step - loss: 0.0641 - val_loss: 0.1126
Epoch 51/10000
46875/46875 [==============================] - 14s 296us/step - loss: 0.0638 - val_loss: 0.1156
Epoch 52/10000
46875/46875 [==============================] - 14s 297us/step - loss: 0.0631 - val_loss: 0.1128
Epoch 53/10000
46875/46875 [==============================] - 14s 297us/step - loss: 0.0633 - val_loss: 0.1164
Epoch 54/10000
4

Epoch 57/10000
46875/46875 [==============================] - 14s 301us/step - loss: 0.0625 - val_loss: 0.1128
Epoch 58/10000
46875/46875 [==============================] - 14s 301us/step - loss: 0.0628 - val_loss: 0.1136
Restoring model weights from the end of the best epoch
Epoch 00058: early stopping
0.3100033573517424


Content based : no interaction influencer/artist taken into account (no embedding for them)

advantage : cold start allowed
disadvantage : interesting info lossed

==> hybrid recommender system

label encoded influencer kind : in production, a category 'Other' can be created to account for potential kinds not present in current dataset.

Grid search

track info ==> proxy of song ID ==> potential leak for this dataset